In [1]:
using BeamLib
using JSON
    
data = JSON.parsefile("manifold_rfsoc_basic_004.json")
display(data["description"])

"Standard 2.4GHz antennas directly attached to ADC-ports A-D on the RFSoC4x2 (roughly 1.75cm between antennas). Complex amplitudes at each position are taken as mean of 10000 samples recorded with active MTS at 100MHz samplerate. Measured at 2.4GHz."

In [6]:
# num_elements x num_azimuths - Matrix
responses = hcat(data["real"] + 1im*data["imag"])

# num_c x num_frequencies x num_azimuths x num_elevations x num_elements
responses = reshape(transpose(responses), 1, 1, size(responses, 2), 1, size(responses, 1))

# create SampledManifold
a = SampledArrayManifold(data["c"], data["frequencies"], AzEl(data["azimuths"]'), responses)

a(2.45e9, deg2rad.([30, 60])')

UndefVarError: UndefVarError: `responses` not defined in `BeamLib`
Suggestion: check for spelling errors or missing imports.